# Extraction de statistiques des joueurs  de football d'eurpe depuis le siteweb fbref

# 1. Europa_league 

1. Ce script est conçu pour être exécuté en local, et il te dira simplement si le tableau est présent ou non. Si ce script affiche "Non trouvé", c’est probablement parce que FBref place le tableau dans un commentaire HTML ou bloque les requêtes sans User-Agent.

In [1]:
# Importation des bibliothèques nécessaires
import requests  # Pour envoyer des requêtes HTTP et récupérer le contenu des pages web
from bs4 import BeautifulSoup  # Pour parser et analyser le code HTML

# URL de la page FBref contenant les statistiques de la Ligue Europa
url = 'https://fbref.com/en/comps/19/stats/Europa-League-Stats'

# Envoi de la requête GET à la page cible
response = requests.get(url)

# Analyse du contenu HTML de la page à l'aide de BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Recherche du tableau HTML contenant les statistiques standards, identifié par son ID "stats_standard"
table = soup.find("table", {"id": "stats_standard"})

# Vérification de la présence du tableau et affichage du résultat
print("Trouvé" if table else "Non trouvé")

Non trouvé


2. L’extraction de tableau constituer les statistics des joueurs d'europa league caché dans les commentaires HTML de FBref.

In [2]:
import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd

# 1. URL de la page FBref contenant les statistiques de la Ligue Europa
url = 'https://fbref.com/en/comps/19/stats/Europa-League-Stats'

# 2. Envoyer une requête GET pour récupérer le contenu HTML de la page
response = requests.get(url)

# 3. Parser le contenu HTML avec BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# 4. Extraire tous les commentaires HTML, car le tableau est souvent caché dans les commentaires
comments = soup.find_all(string=lambda text: isinstance(text, Comment))

# 5. Initialiser la variable df qui contiendra le DataFrame final
df = None

# 6. Parcourir les commentaires pour chercher le tableau avec l'ID "stats_standard"
for comment in comments:
    if 'stats_standard' in comment:  # Vérifie si le commentaire contient le tableau recherché
        # Parser le commentaire comme un HTML séparé
        comment_soup = BeautifulSoup(comment, 'html.parser')
        # Chercher le tableau avec l'ID "stats_standard"
        table = comment_soup.find('table', {'id': 'stats_standard'})
        if table:
            # Convertir le tableau HTML en DataFrame Pandas
            df = pd.read_html(str(table))[0]
            break  # Sortir de la boucle dès qu'on trouve le tableau

# 7. Afficher les données ou indiquer que le tableau est introuvable
if df is not None:
    print(df.head())  # Affiche les 5 premières lignes du DataFrame
    # Enregistrer les données dans un fichier CSV (optionnel)
    df.to_csv("europa_league_stats.csv", index=False)
else:
    print("Tableau non trouvé")


  Unnamed: 0_level_0   Unnamed: 1_level_0 Unnamed: 2_level_0  \
                  Rk               Player             Nation   
0                  1  Mohamed Abdelmoneim             eg EGY   
1                  2             Ali Abdi             tn TUN   
2                  3        Taha Abdi Ali             se SWE   
3                  4        Jalal Abdulai             gh GHA   
4                  5      Saud Abdulhamid             sa KSA   

  Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0 Unnamed: 6_level_0  \
                 Pos              Squad                Age               Born   
0                 DF            fr Nice                 25               1999   
1              DF,MF            fr Nice                 30               1993   
2                 FW           se Malmö                 26               1998   
3                 FW        se Elfsborg                 19               2005   
4                 DF            it Roma                 25       

In [3]:
df.columns = df.columns.get_level_values(-1)

In [4]:
# Suppression des lignes où la colonne 'Rk' contient 'Rk' (en tant que texte)
df = df[df["Rk"] != "Rk"]

In [5]:
df.shape

(917, 37)

In [6]:
#la somme des valeurs nulls
df.isnull().sum()

Rk          0
Player      0
Nation      4
Pos         0
Squad       0
Age         2
Born        2
MP          0
Starts      0
Min         0
90s         0
Gls         0
Ast         0
G+A         0
G-PK        0
PK          0
PKatt       0
CrdY        0
CrdR        0
xG          0
npxG        0
xAG         0
npxG+xAG    0
PrgC        0
PrgP        0
PrgR        0
Gls         0
Ast         0
G+A         0
G-PK        0
G+A-PK      0
xG          1
xAG         1
xG+xAG      1
npxG        1
npxG+xAG    1
Matches     0
dtype: int64

In [10]:
print(df['Squad'].unique())

['Nice' 'Malmö' 'Elfsborg' 'Roma' 'Lyon' 'Ferencváros' 'Qarabağ'
 'AZ Alkmaar' 'Maccabi Tel Aviv' 'Anderlecht' 'Viktoria Plzeň'
 'Athletic Club' 'Real Sociedad' 'Tottenham' 'Fenerbahçe' 'Galatasaray'
 'Hoffenheim' 'Ajax' 'FCSB' 'Ludogorets' 'Manchester Utd' 'Eint Frankfurt'
 'Midtjylland' 'Bodø/Glimt' 'Dynamo Kyiv' 'Olympiacos' 'Braga' 'Porto'
 'Rangers' 'PAOK' 'Lazio' 'FK Rīgas FS' 'Twente' 'Slavia Prague'
 'Union SG' 'Beşiktaş']


In [9]:
# Nettoyer toute la colonne 'Nation' pour ne garder que le pays (ex: "EGY")
df['Nation'] = df['Nation'].astype(str).apply(lambda x: x.split()[-1])

# Nettoyer toute la colonne 'Squad' pour ne garder que le club (ex: "Nice")
df['Squad'] = df['Squad'].astype(str).apply(lambda x: ' '.join(x.split()[1:]))

# Supprimer des colonnes non souhaitées
df = df.drop(columns=["Born", "Matches"])

In [39]:
df.fillna(0)
df

,Rk,Player,Nation,Pos,Squad,Age,MP,Starts,Min,90s,...,Gls,Ast,G+A,G-PK,G+A-PK,xG,xAG,xG+xAG,npxG,npxG+xAG
0,1,Mohamed Abdelmoneim,EGY,DF,Nice,25,4,4,360,4.0,...,0.00,0.00,0.00,0.00,0.00,0.03,0.02,0.05,0.03,0.05
1,2,Ali Abdi,TUN,"DF,MF",Nice,30,4,1,136,1.5,...,0.00,0.00,0.00,0.00,0.00,0.00,0.16,0.16,0.00,0.16
2,3,Taha Abdi Ali,SWE,FW,Malmö,26,7,3,288,3.2,...,0.31,0.00,0.31,0.31,0.31,0.23,0.03,0.26,0.23,0.26
3,4,Jalal Abdulai,GHA,FW,Elfsborg,19,5,2,241,2.7,...,0.00,0.00,0.00,0.00,0.00,0.25,0.03,0.28,0.25,0.28
4,5,Saud Abdulhamid,KSA,DF,Roma,25,4,2,175,1.9,...,0.51,0.00,0.51,0.51,0.51,0.20,0.31,0.51,0.20,0.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,913,Jeroen Zoet,NED,GK,AZ Alkmaar,33,1,1,90,1.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
949,914,Abdellah Zoubir,FRA,"FW,MF",Qarabağ,32,8,8,689,7.7,...,0.13,0.13,0.26,0.13,0.26,0.11,0.10,0.21,0.11,0.21
950,915,Igor Zubeldia,ESP,DF,Real Sociedad,27,8,8,675,7.5,...,0.00,0.00,0.00,0.00,0.00,0.03,0.02,0.05,0.03,0.05
951,916,Martín Zubimendi,ESP,MF,Real Sociedad,25,8,7,602,6.7,...,0.00,0.00,0.00,0.00,0.00,0.04,0.01,0.05,0.04,0.05


In [40]:
import os
# Définir le chemin de destination
dossier_destination = r"C:\Users\LENOVO\Documents\Projet Analyse du web\Final Document"
df.to_csv(os.path.join(dossier_destination, "Europa_league_final_dataset.csv"), index=False)

# 2. Les cinq grandes ligues d'europe

1. Si le script affiche "Trouvé", cela signifie que le tableau avec l’ID stats_standard a bien été localisé dans le code HTML (ou dans un commentaire HTML) et que tu peux maintenant accéder aux données.

In [12]:
import requests
from bs4 import BeautifulSoup

url = 'https://fbref.com/en/comps/Big5/stats/players/Big-5-European-Leagues-Stats'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Rechercher le tableau avec l'ID "stats_standard"
table = soup.find("table", {"id": "stats_standard"})
print("Trouvé" if table else "Non trouvé")


Trouvé


 2. extraction des statistiques des joueurs des cinq grandes ligues européennes depuis FBref,

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 1. URL de la page contenant les statistiques des joueurs des 5 grandes ligues européennes
url = 'https://fbref.com/en/comps/Big5/stats/players/Big-5-European-Leagues-Stats'

# 2. Envoyer une requête HTTP GET pour récupérer le contenu HTML de la page
response = requests.get(url)

# 3. Analyser le contenu HTML avec BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# 4. Rechercher le tableau HTML avec l'ID "stats_standard" qui contient les statistiques des joueurs
table = soup.find("table", {"id": "stats_standard"})

# 5. Convertir ce tableau HTML en un DataFrame Pandas pour manipulation facile des données
df1 = pd.read_html(str(table))[0]

# 6. Afficher les 5 premières lignes du DataFrame pour vérification
print(df1.head())

# 7. Optionnel : sauvegarder les données dans un fichier CSV local
df1.to_csv("big5_european_leagues_players_stats.csv", index=False)


  Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
                  Rk             Player             Nation                Pos   
0                  1         Max Aarons            eng ENG                 DF   
1                  2         Max Aarons            eng ENG              DF,MF   
2                  3     Rodrigo Abajas             es ESP                 DF   
3                  4     James Abankwah             ie IRL              DF,MF   
4                  5  Keyliane Abdallah             fr FRA                 FW   

  Unnamed: 4_level_0  Unnamed: 5_level_0 Unnamed: 6_level_0  \
               Squad                Comp                Age   
0        Bournemouth  eng Premier League             25-140   
1           Valencia          es La Liga             25-140   
2           Valencia          es La Liga             22-012   
3            Udinese          it Serie A             21-128   
4          Marseille          fr Ligue 1             1

In [14]:
df1.shape

(2956, 38)

In [15]:
df1.isnull().sum()

Unnamed: 0_level_0   Rk          0
Unnamed: 1_level_0   Player      0
Unnamed: 2_level_0   Nation      6
Unnamed: 3_level_0   Pos         0
Unnamed: 4_level_0   Squad       0
Unnamed: 5_level_0   Comp        0
Unnamed: 6_level_0   Age         7
Unnamed: 7_level_0   Born        7
Playing Time         MP          0
                     Starts      0
                     Min         0
                     90s         0
Performance          Gls         0
                     Ast         0
                     G+A         0
                     G-PK        0
                     PK          0
                     PKatt       0
                     CrdY        0
                     CrdR        0
Expected             xG          0
                     npxG        0
                     xAG         0
                     npxG+xAG    0
Progression          PrgC        0
                     PrgP        0
                     PrgR        0
Per 90 Minutes       Gls         0
                    

In [17]:
df1

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                     Rk             Player             Nation   
0                     1         Max Aarons            eng ENG   
1                     2         Max Aarons            eng ENG   
2                     3     Rodrigo Abajas             es ESP   
3                     4     James Abankwah             ie IRL   
4                     5  Keyliane Abdallah             fr FRA   
...                 ...                ...                ...   
2951               2839       Edhy Zuliani             it ITA   
2952               2840   Szymon Żurkowski             pl POL   
2953               2841    Martin Ødegaard             no NOR   
2954               2842        Milan Đurić             ba BIH   
2955               2843        Milan Đurić             ba BIH   

     Unnamed: 3_level_0 Unnamed: 4_level_0  Unnamed: 5_level_0  \
                    Pos              Squad                Comp   
0                    DF        Bournemouth  eng Premier League   
1                 DF,MF           Valencia          es La Liga   
2                    DF           Valencia          es La Liga   
3                 DF,MF            Udinese          it Serie A   
4                    FW          Marseille          fr Ligue 1   
...                 ...                ...                 ...   
2951                 DF           Toulouse          fr Ligue 1   
2952                 MF             Empoli          it Serie A   
2953                 MF            Arsenal  eng Premier League   
2954                 FW              Monza          it Serie A   
2955                 FW              Parma          it Serie A   

     Unnamed: 6_level_0 Unnamed: 7_level_0 Playing Time         ...  \
                    Age               Born           MP Starts  ...   
0                25-140               2000            3      1  ...   
1                25-140               2000            4      1  ...   
2                22-012               2003            1      1  ...   
3                21-128               2004            6      0  ...   
4                19-049               2006            1      0  ...   
...                 ...                ...          ...    ...  ...   
2951             20-286               2004            1      0  ...   
2952             27-241               1997            5      0  ...   
2953             26-158               1998           29     26  ...   
2954             35-002               1990           18     13  ...   
2955             35-002               1990            9      3  ...   

     Per 90 Minutes                                                       \
                Ast   G+A  G-PK G+A-PK    xG   xAG xG+xAG  npxG npxG+xAG   
0              0.00  0.00  0.00   0.00  0.00  0.00   0.00  0.00     0.00   
1              0.00  0.00  0.00   0.00  0.00  0.02   0.02  0.00     0.02   
2              0.00  0.00  0.00   0.00  0.10  0.00   0.10  0.10     0.10   
3              0.00  0.00  0.00   0.00  0.06  0.00   0.06  0.06     0.06   
4              0.00  0.00  0.00   0.00  0.00  0.00   0.00  0.00     0.00   
...             ...   ...   ...    ...   ...   ...    ...   ...      ...   
2951           0.00  0.00  0.00   0.00  0.00  0.00   0.00  0.00     0.00   
2952           0.00  0.00  0.00   0.00  0.07  0.00   0.07  0.07     0.07   
2953           0.31  0.39  0.04   0.35  0.19  0.21   0.40  0.15     0.37   
2954           0.07  0.35  0.28   0.35  0.21  0.08   0.28  0.21     0.28   
2955           0.00  0.34  0.34   0.34  0.08  0.00   0.08  0.08     0.08   

     Unnamed: 37_level_0  
                 Matches  
0                Matches  
1                Matches  
2                Matches  
3                Matches  
4                Matches  
...                  ...  
2951             Matches  
2952             Matches  
2953             Matches  
2954             Matches  
2955             Matches  

[2956 rows x 38 columns]

In [27]:
print(df1['Squad'].unique())

['Bournemouth' 'Valencia' 'Udinese' 'Marseille' 'Saint-Étienne' 'Angers'
 'Nice' 'Roma' 'Osasuna' 'Getafe' 'Como' 'Nantes' 'Lyon' 'Toulouse'
 'Alavés' 'Milan' 'Genoa' 'Inter' 'Chelsea' 'Sevilla' 'Montpellier'
 'Torino' 'Squad' 'Freiburg' 'Reims' 'Dortmund' 'Brighton' 'Leverkusen'
 'Fiorentina' 'Betis' 'Juventus' 'Bologna' 'St. Pauli' 'Wolves'
 'Crystal Palace' 'Lens' 'Athletic Club' 'Werder Bremen' 'Espanyol'
 'Real Madrid' 'Real Sociedad' 'Rennes' 'Valladolid' 'Celta Vigo'
 "Nott'ham Forest" 'Hellas Verona' 'Brentford' 'Brest' 'Manchester City'
 'Villarreal' 'Monaco' 'Auxerre' 'Monza' 'Hoffenheim' 'Lille'
 'Southampton' 'Everton' 'Liverpool' 'Girona' 'Newcastle Utd' 'Parma'
 'Venezia' 'Leganés' 'West Ham' 'Atlético Madrid' 'Leicester City'
 'Manchester Utd' 'Eint Frankfurt' 'Mainz 05' 'Strasbourg' 'Wolfsburg'
 'Fulham' 'Empoli' 'Barcelona' 'Holstein Kiel' 'Mallorca' 'Aston Villa'
 'Paris S-G' 'Cagliari' 'Tottenham' 'Le Havre' 'Bayern Munich' 'Bochum'
 'Las Palmas' 'Atalanta' 'RB Leipz

In [23]:
# Suppression du multi-index des colonnes (niveau "Unnamed")
df1.columns =df1.columns.get_level_values(-1)


# Nettoyage de la colonne 'Nation' : garde uniquement le code pays (ex: "EGY")
df1['Nation'] = df1['Nation'].apply(lambda x: str(x).split()[-1])

# Nettoyage de la colonne 'Comp' : garde uniquement le nom de la compétition (ex: "Premier League")
df1['Comp'] = df1['Comp'].apply(lambda x: str(x).split(' ', 1)[-1])

# Nettoyage de la colonne 'Age' : garde uniquement l'âge avant le tiret, converti en entier
df1['Age'] = df1['Age'].apply(
    lambda x: int(str(x).split('-')[0]) if pd.notna(x) and '-' in str(x) else None
)


In [26]:
# Supprimer les lignes qui contiennent "Rk" dans la colonne 'Rk' ou dans une autre colonne selon ta structure
data_final = df1[df1["Rk"] != "Rk"]

In [42]:
data_final.isnull().sum()
data_final.fillna(0)

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,Ast,G+A,G-PK,G+A-PK,xG,xAG,xG+xAG,npxG,npxG+xAG,Matches
0,1,Max Aarons,ENG,DF,Bournemouth,Premier League,25.0,2000,3,1,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
1,2,Max Aarons,ENG,"DF,MF",Valencia,La Liga,25.0,2000,4,1,...,0.00,0.00,0.00,0.00,0.00,0.02,0.02,0.00,0.02,Matches
2,3,Rodrigo Abajas,ESP,DF,Valencia,La Liga,22.0,2003,1,1,...,0.00,0.00,0.00,0.00,0.10,0.00,0.10,0.10,0.10,Matches
3,4,James Abankwah,IRL,"DF,MF",Udinese,Serie A,21.0,2004,6,0,...,0.00,0.00,0.00,0.00,0.06,0.00,0.06,0.06,0.06,Matches
4,5,Keyliane Abdallah,FRA,FW,Marseille,Ligue 1,19.0,2006,1,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2951,2839,Edhy Zuliani,ITA,DF,Toulouse,Ligue 1,20.0,2004,1,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
2952,2840,Szymon Żurkowski,POL,MF,Empoli,Serie A,27.0,1997,5,0,...,0.00,0.00,0.00,0.00,0.07,0.00,0.07,0.07,0.07,Matches
2953,2841,Martin Ødegaard,NOR,MF,Arsenal,Premier League,26.0,1998,29,26,...,0.31,0.39,0.04,0.35,0.19,0.21,0.40,0.15,0.37,Matches
2954,2842,Milan Đurić,BIH,FW,Monza,Serie A,35.0,1990,18,13,...,0.07,0.35,0.28,0.35,0.21,0.08,0.28,0.21,0.28,Matches


In [43]:
print(data_final['Squad'].unique())

['Bournemouth' 'Valencia' 'Udinese' 'Marseille' 'Saint-Étienne' 'Angers'
 'Nice' 'Roma' 'Osasuna' 'Getafe' 'Como' 'Nantes' 'Lyon' 'Toulouse'
 'Alavés' 'Milan' 'Genoa' 'Inter' 'Chelsea' 'Sevilla' 'Montpellier'
 'Torino' 'Freiburg' 'Reims' 'Dortmund' 'Brighton' 'Leverkusen'
 'Fiorentina' 'Betis' 'Juventus' 'Bologna' 'St. Pauli' 'Wolves'
 'Crystal Palace' 'Lens' 'Athletic Club' 'Werder Bremen' 'Espanyol'
 'Real Madrid' 'Real Sociedad' 'Rennes' 'Valladolid' 'Celta Vigo'
 "Nott'ham Forest" 'Hellas Verona' 'Brentford' 'Brest' 'Manchester City'
 'Villarreal' 'Monaco' 'Auxerre' 'Monza' 'Hoffenheim' 'Lille'
 'Southampton' 'Everton' 'Liverpool' 'Girona' 'Newcastle Utd' 'Parma'
 'Venezia' 'Leganés' 'West Ham' 'Atlético Madrid' 'Leicester City'
 'Manchester Utd' 'Eint Frankfurt' 'Mainz 05' 'Strasbourg' 'Wolfsburg'
 'Fulham' 'Empoli' 'Barcelona' 'Holstein Kiel' 'Mallorca' 'Aston Villa'
 'Paris S-G' 'Cagliari' 'Tottenham' 'Le Havre' 'Bayern Munich' 'Bochum'
 'Las Palmas' 'Atalanta' 'RB Leipzig' 'Ray

In [44]:
import os
# Définir le chemin de destination
dossier_destination = r"C:\Users\LENOVO\Documents\Projet Analyse du web\Final Document"
data_final.to_csv(os.path.join(dossier_destination, "Big5_Europian_leagues_final_dataset.csv"), index=False)

# 3. Champions league

In [28]:
import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd

# 1. URL de la page FBref avec les stats standards de la Ligue des Champions
url = 'https://fbref.com/en/comps/8/stats/Champions-League-Stats'

# 2. Télécharger la page HTML
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# 3. Extraire les commentaires HTML (les tableaux sont souvent là-dedans)
comments = soup.find_all(string=lambda text: isinstance(text, Comment))

# 4. Chercher le tableau 'stats_standard' dans les commentaires
df2 = None
for comment in comments:
    if 'stats_standard' in comment:
        comment_soup = BeautifulSoup(comment, 'html.parser')
        table = comment_soup.find('table', {'id': 'stats_standard'})
        if table:
            df2 = pd.read_html(str(table))[0]
            break

# 5. Vérifier et sauvegarder
if df2 is not None:
    print(df2.head())
    df2.to_csv("champions_league_standard_stats.csv", index=False)
else:
    print("❌ Tableau 'stats_standard' non trouvé.")


  Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
                  Rk             Player             Nation                Pos   
0                  1      Tammy Abraham            eng ENG                 FW   
1                  2   Francesco Acerbi             it ITA                 DF   
2                  3       Arijan Ademi             mk MKD                 MF   
3                  4      Karim Adeyemi             de GER              FW,MF   
4                  5         Amine Adli             ma MAR              MF,FW   

  Unnamed: 4_level_0 Unnamed: 5_level_0 Unnamed: 6_level_0 Playing Time  \
               Squad                Age               Born           MP   
0           it Milan             27-234               1997            9   
1           it Inter             37-103               1988            8   
2   hr Dinamo Zagreb             33-360               1991            6   
3        de Dortmund             23-126               200

In [29]:
df2.shape

(921, 37)

In [30]:
# Nettoyage des colonnes pour supprimer les niveaux Unnamed:
df2.columns = df2.columns.get_level_values(-1)

# Suppression des lignes où la colonne 'Rk' contient 'Rk' (en tant que texte)
df2 = df2[df2["Rk"] != "Rk"]
df2

,Rk,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,...,Ast,G+A,G-PK,G+A-PK,xG,xAG,xG+xAG,npxG,npxG+xAG,Matches
0,1,Tammy Abraham,eng ENG,FW,it Milan,27-234,1997,9,2,301,...,0.30,0.90,0.60,0.90,0.36,0.20,0.56,0.36,0.56,Matches
1,2,Francesco Acerbi,it ITA,DF,it Inter,37-103,1988,8,7,680,...,0.13,0.26,0.13,0.26,0.07,0.09,0.16,0.07,0.16,Matches
2,3,Arijan Ademi,mk MKD,MF,hr Dinamo Zagreb,33-360,1991,6,4,287,...,0.00,0.00,0.00,0.00,0.20,0.08,0.28,0.20,0.28,Matches
3,4,Karim Adeyemi,de GER,"FW,MF",de Dortmund,23-126,2002,10,9,685,...,0.13,0.79,0.66,0.79,0.27,0.21,0.48,0.27,0.48,Matches
4,5,Amine Adli,ma MAR,"MF,FW",de Leverkusen,25-014,2000,5,2,210,...,0.00,0.00,0.00,0.00,0.08,0.07,0.15,0.08,0.15,Matches
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916,882,Tanguy Zoukrou,fr FRA,DF,ch Young Boys,22-017,2003,3,3,270,...,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.01,Matches
917,883,Oleksandr Zubkov,ua UKR,"FW,MF",ua Shakhtar,28-294,1996,8,7,532,...,0.17,0.51,0.34,0.51,0.22,0.17,0.40,0.22,0.40,Matches
918,884,Lovro Zvonarek,hr CRO,"MF,FW",at Sturm Graz,20-016,2005,6,0,140,...,0.00,0.00,0.00,0.00,0.22,0.07,0.29,0.22,0.29,Matches
919,885,Martin Ødegaard,no NOR,MF,eng Arsenal,26-158,1998,11,9,797,...,0.11,0.45,0.34,0.45,0.21,0.17,0.38,0.21,0.38,Matches


In [31]:
print(df2['Squad'].unique())

['it Milan' 'it Inter' 'hr Dinamo Zagreb' 'de Dortmund' 'de Leverkusen'
 'it Juventus' 'it Bologna' 'at Sturm Graz' 'fr Brest'
 'eng Manchester City' 'fr Monaco' 'fr Lille' 'pt Benfica'
 'es Real Madrid' 'eng Liverpool' 'es Atlético Madrid' 'ch Young Boys'
 'pt Sporting CP' 'es Barcelona' 'eng Aston Villa' 'fr Paris S-G'
 'es Girona' 'ua Shakhtar' 'de Bayern Munich' 'nl PSV Eindhoven'
 'at RB Salzburg' 'sk Slovan Bratislava' 'de RB Leipzig' 'nl Feyenoord'
 'it Atalanta' 'sct Celtic' 'de Stuttgart' 'cz Sparta Prague'
 'eng Arsenal' 'rs Red Star' 'be Club Brugge']


In [32]:
# Nettoyer toute la colonne 'Nation' pour ne garder que le pays (ex: "EGY")
df2['Nation'] = df2['Nation'].astype(str).apply(lambda x: x.split()[-1])

# Nettoyer toute la colonne 'Squad' pour ne garder que le club (ex: "Nice")
df2['Squad'] = df2['Squad'].astype(str).apply(lambda x: ' '.join(x.split()[1:]))

# Supprimer des colonnes non souhaitées
df2 = df2.drop(columns=["Born", "Matches"])
df2

,Rk,Player,Nation,Pos,Squad,Age,MP,Starts,Min,90s,...,Gls,Ast,G+A,G-PK,G+A-PK,xG,xAG,xG+xAG,npxG,npxG+xAG
0,1,Tammy Abraham,ENG,FW,Milan,27-234,9,2,301,3.3,...,0.60,0.30,0.90,0.60,0.90,0.36,0.20,0.56,0.36,0.56
1,2,Francesco Acerbi,ITA,DF,Inter,37-103,8,7,680,7.6,...,0.13,0.13,0.26,0.13,0.26,0.07,0.09,0.16,0.07,0.16
2,3,Arijan Ademi,MKD,MF,Dinamo Zagreb,33-360,6,4,287,3.2,...,0.00,0.00,0.00,0.00,0.00,0.20,0.08,0.28,0.20,0.28
3,4,Karim Adeyemi,GER,"FW,MF",Dortmund,23-126,10,9,685,7.6,...,0.66,0.13,0.79,0.66,0.79,0.27,0.21,0.48,0.27,0.48
4,5,Amine Adli,MAR,"MF,FW",Leverkusen,25-014,5,2,210,2.3,...,0.00,0.00,0.00,0.00,0.00,0.08,0.07,0.15,0.08,0.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916,882,Tanguy Zoukrou,FRA,DF,Young Boys,22-017,3,3,270,3.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.01
917,883,Oleksandr Zubkov,UKR,"FW,MF",Shakhtar,28-294,8,7,532,5.9,...,0.34,0.17,0.51,0.34,0.51,0.22,0.17,0.40,0.22,0.40
918,884,Lovro Zvonarek,CRO,"MF,FW",Sturm Graz,20-016,6,0,140,1.6,...,0.00,0.00,0.00,0.00,0.00,0.22,0.07,0.29,0.22,0.29
919,885,Martin Ødegaard,NOR,MF,Arsenal,26-158,11,9,797,8.9,...,0.34,0.11,0.45,0.34,0.45,0.21,0.17,0.38,0.21,0.38


In [33]:
print(df2['Squad'].unique())

['Milan' 'Inter' 'Dinamo Zagreb' 'Dortmund' 'Leverkusen' 'Juventus'
 'Bologna' 'Sturm Graz' 'Brest' 'Manchester City' 'Monaco' 'Lille'
 'Benfica' 'Real Madrid' 'Liverpool' 'Atlético Madrid' 'Young Boys'
 'Sporting CP' 'Barcelona' 'Aston Villa' 'Paris S-G' 'Girona' 'Shakhtar'
 'Bayern Munich' 'PSV Eindhoven' 'RB Salzburg' 'Slovan Bratislava'
 'RB Leipzig' 'Feyenoord' 'Atalanta' 'Celtic' 'Stuttgart' 'Sparta Prague'
 'Arsenal' 'Red Star' 'Club Brugge']


In [34]:
# Nettoyage de la colonne 'Age' : garde uniquement le nombre avant le tiret, converti en entier
df2['Age'] = df2['Age'].apply(lambda x: int(str(x).split('-')[0]))

In [35]:
df2.isnull().sum()

Rk          0
Player      0
Nation      0
Pos         0
Squad       0
Age         0
MP          0
Starts      0
Min         0
90s         0
Gls         0
Ast         0
G+A         0
G-PK        0
PK          0
PKatt       0
CrdY        0
CrdR        0
xG          0
npxG        0
xAG         0
npxG+xAG    0
PrgC        0
PrgP        0
PrgR        0
Gls         0
Ast         0
G+A         0
G-PK        0
G+A-PK      0
xG          0
xAG         0
xG+xAG      0
npxG        0
npxG+xAG    0
dtype: int64

In [36]:
df2.shape

(886, 35)

In [37]:
df2

,Rk,Player,Nation,Pos,Squad,Age,MP,Starts,Min,90s,...,Gls,Ast,G+A,G-PK,G+A-PK,xG,xAG,xG+xAG,npxG,npxG+xAG
0,1,Tammy Abraham,ENG,FW,Milan,27,9,2,301,3.3,...,0.60,0.30,0.90,0.60,0.90,0.36,0.20,0.56,0.36,0.56
1,2,Francesco Acerbi,ITA,DF,Inter,37,8,7,680,7.6,...,0.13,0.13,0.26,0.13,0.26,0.07,0.09,0.16,0.07,0.16
2,3,Arijan Ademi,MKD,MF,Dinamo Zagreb,33,6,4,287,3.2,...,0.00,0.00,0.00,0.00,0.00,0.20,0.08,0.28,0.20,0.28
3,4,Karim Adeyemi,GER,"FW,MF",Dortmund,23,10,9,685,7.6,...,0.66,0.13,0.79,0.66,0.79,0.27,0.21,0.48,0.27,0.48
4,5,Amine Adli,MAR,"MF,FW",Leverkusen,25,5,2,210,2.3,...,0.00,0.00,0.00,0.00,0.00,0.08,0.07,0.15,0.08,0.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916,882,Tanguy Zoukrou,FRA,DF,Young Boys,22,3,3,270,3.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.01
917,883,Oleksandr Zubkov,UKR,"FW,MF",Shakhtar,28,8,7,532,5.9,...,0.34,0.17,0.51,0.34,0.51,0.22,0.17,0.40,0.22,0.40
918,884,Lovro Zvonarek,CRO,"MF,FW",Sturm Graz,20,6,0,140,1.6,...,0.00,0.00,0.00,0.00,0.00,0.22,0.07,0.29,0.22,0.29
919,885,Martin Ødegaard,NOR,MF,Arsenal,26,11,9,797,8.9,...,0.34,0.11,0.45,0.34,0.45,0.21,0.17,0.38,0.21,0.38


In [38]:
import os
# Définir le chemin de destination
dossier_destination = r"C:\Users\LENOVO\Documents\Projet Analyse du web\Final Document"
df2.to_csv(os.path.join(dossier_destination, "Champions_league_final_dataset.csv"), index=False)

# 4. Conference League

## L’extraction des données statistiques de la Conference League 2024-2025

In [46]:
# Importation des bibliothèques nécessaires
import requests                        # Pour faire des requêtes HTTP
from bs4 import BeautifulSoup, Comment # Pour parser le HTML et lire les commentaires HTML
import pandas as pd                    # Pour manipuler les données tabulaires (DataFrame)

# 1. Définir l’URL de la page contenant les statistiques de la Conference League
url = 'https://fbref.com/en/comps/882/stats/Conference-League-Stats'

# 2. Envoyer une requête GET pour récupérer le contenu HTML de la page
response = requests.get(url)

# 3. Parser le contenu HTML avec BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# 4. Extraire tous les commentaires HTML présents dans la page
# (FBref place certains tableaux importants à l’intérieur de commentaires HTML)
comments = soup.find_all(string=lambda text: isinstance(text, Comment))

# 5. Initialiser une variable pour stocker le DataFrame
dfc = None

# 6. Parcourir chaque commentaire HTML pour trouver celui contenant le tableau "stats_standard"
for comment in comments:
    if 'stats_standard' in comment:
        # Reparser le commentaire comme un fragment HTML
        comment_soup = BeautifulSoup(comment, 'html.parser')
        
        # Rechercher le tableau ayant l'ID "stats_standard"
        table = comment_soup.find('table', {'id': 'stats_standard'})
        
        # Si le tableau est trouvé, le convertir en DataFrame Pandas
        if table:
            dfc = pd.read_html(str(table))[0]  # Extraction du premier tableau trouvé
            break  # Arrêter la recherche après avoir trouvé le tableau


In [48]:
# Afficher les noms des colonnes
dfc.columns

MultiIndex([( 'Unnamed: 0_level_0',       'Rk'),
            ( 'Unnamed: 1_level_0',   'Player'),
            ( 'Unnamed: 2_level_0',   'Nation'),
            ( 'Unnamed: 3_level_0',      'Pos'),
            ( 'Unnamed: 4_level_0',    'Squad'),
            ( 'Unnamed: 5_level_0',      'Age'),
            ( 'Unnamed: 6_level_0',     'Born'),
            (       'Playing Time',       'MP'),
            (       'Playing Time',   'Starts'),
            (       'Playing Time',      'Min'),
            (       'Playing Time',      '90s'),
            (        'Performance',      'Gls'),
            (        'Performance',      'Ast'),
            (        'Performance',      'G+A'),
            (        'Performance',     'G-PK'),
            (        'Performance',       'PK'),
            (        'Performance',    'PKatt'),
            (        'Performance',     'CrdY'),
            (        'Performance',     'CrdR'),
            (           'Expected',       'xG'),
            (       

In [50]:
# Afficher les dimensions du DataFrame : (nombre de lignes, nombre de colonnes)
dfc

Unnamed: 0_level_0   Unnamed: 1_level_0 Unnamed: 2_level_0  \
                    Rk               Player             Nation   
0                    1         David Abagna             gh GHA   
1                    2  Sławomir Abramowicz             pl POL   
2                    3          Francis Abu             gh GHA   
3                    4    Joshua Acheampong            eng ENG   
4                    5        Elias Achouri             tn TUN   
..                 ...                  ...                ...   
912                878      Aron Þrándarson             is ISL   
913                879          Pavle Đajić             ba BIH   
914                880         Ifet Đakovac             ba BIH   
915                881      Mateja Đorđević             rs SRB   
916                882        Filip Đuričić             rs SRB   

    Unnamed: 3_level_0   Unnamed: 4_level_0 Unnamed: 5_level_0  \
                   Pos                Squad                Age   
0                MF,FW             cy APOEL             26-257   
1                   GK       pl Jagiellonia             20-349   
2                   MF     be Cercle Brugge             24-027   
3                   DF          eng Chelsea             19-019   
4                FW,MF     dk FC Copenhagen             26-103   
..                 ...                  ...                ...   
912              MF,FW                is KV             30-195   
913              DF,FW  ba Borac Banja Luka             18-294   
914              MF,FW     rs TSC Bačka Top             27-170   
915                 DF     rs TSC Bačka Top             22-127   
916              FW,MF     gr Panathinaikos             33-114   

    Unnamed: 6_level_0 Playing Time               ... Per 90 Minutes        \
                  Born           MP Starts   Min  ...            Ast   G+A   
0                 1998            8      7   560  ...           0.00  0.48   
1                 2004           12     12  1080  ...           0.00  0.00   
2                 2001            7      7   554  ...           0.16  0.16   
3                 2006            7      7   585  ...           0.00  0.00   
4                 1999            6      4   336  ...           0.00  0.00   
..                 ...          ...    ...   ...  ...            ...   ...   
912               1994            7      7   465  ...           0.00  0.00   
913               2006            2      0    32  ...           0.00  0.00   
914               1997            6      5   440  ...           0.20  0.61   
915               2003            7      7   629  ...           0.00  0.00   
916               1992           10      9   714  ...           0.25  0.50   

                                                   Unnamed: 36_level_0  
     G-PK G+A-PK    xG   xAG xG+xAG  npxG npxG+xAG             Matches  
0    0.48   0.48  0.37  0.02   0.39  0.37     0.39             Matches  
1    0.00   0.00  0.00  0.00   0.00  0.00     0.00             Matches  
2    0.00   0.16  0.02  0.03   0.05  0.02     0.05             Matches  
3    0.00   0.00  0.03  0.03   0.06  0.03     0.06             Matches  
4    0.00   0.00  0.27  0.07   0.34  0.27     0.34             Matches  
..    ...    ...   ...   ...    ...   ...      ...                 ...  
912  0.00   0.00  0.06  0.03   0.09  0.06     0.09             Matches  
913  0.00   0.00  0.00  0.00   0.00  0.00     0.00             Matches  
914  0.41   0.61  0.48  0.20   0.68  0.28     0.48             Matches  
915  0.00   0.00  0.01  0.01   0.02  0.01     0.02             Matches  
916  0.25   0.50  0.18  0.16   0.35  0.18     0.35             Matches  

[917 rows x 37 columns]

## Nettoyage du dataset de la Conference League 2024-2025

Après avoir ouvert le fichier CSV téléchargé contenant le tableau des statistiques, nous avons constaté que certaines lignes contiennent des en-têtes répétés au lieu de véritables valeurs. Cela est dû à la manière dont le site source structure et affiche les données. Ainsi, la première étape de notre traitement consiste à supprimer ces lignes redondantes

In [53]:
# Filtrer les lignes où la colonne 'Player' ne contient pas le mot "Player",
# car certaines lignes correspondent aux en-têtes répétées et non à des données réelles. 
dfc_clean = dfc[~dfc[('Unnamed: 1_level_0', 'Player')].str.contains("Player", na=False)]

Aussi Les noms des colonnes étaient structurés sur plusieurs niveaux. Pour cette raison, nous allons supprimer le premier niveau et ne conserver que les en-têtes du deuxième niveau, qui contiennent les noms réels des variables.

In [54]:
# Nettoyage des colonnes pour supprimer les niveaux Unnamed:
dfc_clean.columns = dfc_clean.columns.get_level_values(-1)
dfc_clean

,Rk,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,...,Ast,G+A,G-PK,G+A-PK,xG,xAG,xG+xAG,npxG,npxG+xAG,Matches
0,1,David Abagna,gh GHA,"MF,FW",cy APOEL,26-257,1998,8,7,560,...,0.00,0.48,0.48,0.48,0.37,0.02,0.39,0.37,0.39,Matches
1,2,Sławomir Abramowicz,pl POL,GK,pl Jagiellonia,20-349,2004,12,12,1080,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
2,3,Francis Abu,gh GHA,MF,be Cercle Brugge,24-027,2001,7,7,554,...,0.16,0.16,0.00,0.16,0.02,0.03,0.05,0.02,0.05,Matches
3,4,Joshua Acheampong,eng ENG,DF,eng Chelsea,19-019,2006,7,7,585,...,0.00,0.00,0.00,0.00,0.03,0.03,0.06,0.03,0.06,Matches
4,5,Elias Achouri,tn TUN,"FW,MF",dk FC Copenhagen,26-103,1999,6,4,336,...,0.00,0.00,0.00,0.00,0.27,0.07,0.34,0.27,0.34,Matches
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,878,Aron Þrándarson,is ISL,"MF,FW",is KV,30-195,1994,7,7,465,...,0.00,0.00,0.00,0.00,0.06,0.03,0.09,0.06,0.09,Matches
913,879,Pavle Đajić,ba BIH,"DF,FW",ba Borac Banja Luka,18-294,2006,2,0,32,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
914,880,Ifet Đakovac,ba BIH,"MF,FW",rs TSC Bačka Top,27-170,1997,6,5,440,...,0.20,0.61,0.41,0.61,0.48,0.20,0.68,0.28,0.48,Matches
915,881,Mateja Đorđević,rs SRB,DF,rs TSC Bačka Top,22-127,2003,7,7,629,...,0.00,0.00,0.00,0.00,0.01,0.01,0.02,0.01,0.02,Matches


Comme on peut le voir dans ce dataset, la colonne Nation, qui contient les nationalités des joueurs, comporte des valeurs répétées ou combinées, comme par exemple "fr FAR". Pour cela, nous allons supprimer la première partie et ne conserver que la deuxième, "FAR". La même opération sera appliquée à la colonne Squad, qui contient les noms des clubs.

In [55]:
# Nettoyer toute la colonne 'Nation' pour ne garder que le pays (ex: "EGY")
dfc_clean['Nation'] = dfc_clean['Nation'].astype(str).apply(lambda x: x.split()[-1])

# Nettoyer toute la colonne 'Squad' pour ne garder que le club (ex: "Nice")
dfc_clean['Squad'] = dfc_clean['Squad'].astype(str).apply(lambda x: ' '.join(x.split()[1:]))

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_19028\690357689.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc_clean['Nation'] = dfc_clean['Nation'].astype(str).apply(lambda x: x.split()[-1])
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_19028\690357689.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc_clean['Squad'] = dfc_clean['Squad'].astype(str).apply(lambda x: ' '.join(x.split()[1:]))


In [56]:
print(dfc_clean['Squad'].unique())

['APOEL' 'Jagiellonia' 'Cercle Brugge' 'Chelsea' 'FC Copenhagen'
 'Dinamo Minsk' 'Legia Warsaw' 'Fiorentina' 'Betis' 'Olimpija' 'KV'
 'Astana FK' 'FC Noah' 'St. Gallen' 'Djurgården' 'Başakşehir' 'AC Omonia'
 'Lugano' 'Rapid Wien' 'Petrocub' 'Molde' 'Pafos FC' 'HJK' 'Panathinaikos'
 'Gent' 'Vitória' 'The New Saints' 'Hearts' 'TSC Bačka Top' 'Heidenheim'
 'NK Celje' 'LASK' 'Larne FC' 'Shamrock Rov' 'Mladá Boleslav'
 'Borac Banja Luka']


In [61]:
dfc_clean

,Rk,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,...,Ast,G+A,G-PK,G+A-PK,xG,xAG,xG+xAG,npxG,npxG+xAG,Matches
0,1,David Abagna,GHA,"MF,FW",APOEL,26,1998,8,7,560,...,0.00,0.48,0.48,0.48,0.37,0.02,0.39,0.37,0.39,Matches
1,2,Sławomir Abramowicz,POL,GK,Jagiellonia,20,2004,12,12,1080,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
2,3,Francis Abu,GHA,MF,Cercle Brugge,24,2001,7,7,554,...,0.16,0.16,0.00,0.16,0.02,0.03,0.05,0.02,0.05,Matches
3,4,Joshua Acheampong,ENG,DF,Chelsea,19,2006,7,7,585,...,0.00,0.00,0.00,0.00,0.03,0.03,0.06,0.03,0.06,Matches
4,5,Elias Achouri,TUN,"FW,MF",FC Copenhagen,26,1999,6,4,336,...,0.00,0.00,0.00,0.00,0.27,0.07,0.34,0.27,0.34,Matches
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,878,Aron Þrándarson,ISL,"MF,FW",KV,30,1994,7,7,465,...,0.00,0.00,0.00,0.00,0.06,0.03,0.09,0.06,0.09,Matches
913,879,Pavle Đajić,BIH,"DF,FW",Borac Banja Luka,18,2006,2,0,32,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
914,880,Ifet Đakovac,BIH,"MF,FW",TSC Bačka Top,27,1997,6,5,440,...,0.20,0.61,0.41,0.61,0.48,0.20,0.68,0.28,0.48,Matches
915,881,Mateja Đorđević,SRB,DF,TSC Bačka Top,22,2003,7,7,629,...,0.00,0.00,0.00,0.00,0.01,0.01,0.02,0.01,0.02,Matches


Nous allons également nettoyer la colonne Age afin de ne conserver que le nombre d'années, en supprimant les jours qui y sont associés.

In [60]:
# Nettoyage de la colonne 'Age' : garde uniquement le nombre avant le tiret, converti en entier
dfc_clean['Age'] =dfc_clean['Age'].apply(lambda x: int(str(x).split('-')[0]))

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_19028\3620284782.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc_clean['Age'] =dfc_clean['Age'].apply(lambda x: int(str(x).split('-')[0]))


Nous allons maintenant vérifier s'il y a des valeurs manquantes pour les supprimer.

In [62]:
# verifier si il ya des valeurs manquantes
dfc_clean.isnull().sum()

Rk          0
Player      0
Nation      0
Pos         0
Squad       0
Age         0
Born        0
MP          0
Starts      0
Min         0
90s         0
Gls         0
Ast         0
G+A         0
G-PK        0
PK          0
PKatt       0
CrdY        0
CrdR        0
xG          0
npxG        0
xAG         0
npxG+xAG    0
PrgC        0
PrgP        0
PrgR        0
Gls         0
Ast         0
G+A         0
G-PK        0
G+A-PK      0
xG          4
xAG         4
xG+xAG      4
npxG        4
npxG+xAG    4
Matches     0
dtype: int64

In [65]:
colonnes_avec_nan = ['xG', 'xAG', 'xG+xAG', 'npxG', 'npxG+xAG']
dfc_clean.dropna(subset=colonnes_avec_nan, inplace=True)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_19028\4125649590.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc_clean.dropna(subset=colonnes_avec_nan, inplace=True)


In [66]:
dfc_clean.isnull().sum()

Rk          0
Player      0
Nation      0
Pos         0
Squad       0
Age         0
Born        0
MP          0
Starts      0
Min         0
90s         0
Gls         0
Ast         0
G+A         0
G-PK        0
PK          0
PKatt       0
CrdY        0
CrdR        0
xG          0
npxG        0
xAG         0
npxG+xAG    0
PrgC        0
PrgP        0
PrgR        0
Gls         0
Ast         0
G+A         0
G-PK        0
G+A-PK      0
xG          0
xAG         0
xG+xAG      0
npxG        0
npxG+xAG    0
Matches     0
dtype: int64

Maintenant on va supprimer les colonnes non utiles

In [67]:
# Supprimer des colonnes non souhaitées
dfc_clean= dfc_clean.drop(columns=["Born", "Matches"])

In [68]:
dfc_clean

,Rk,Player,Nation,Pos,Squad,Age,MP,Starts,Min,90s,...,Gls,Ast,G+A,G-PK,G+A-PK,xG,xAG,xG+xAG,npxG,npxG+xAG
0,1,David Abagna,GHA,"MF,FW",APOEL,26,8,7,560,6.2,...,0.48,0.00,0.48,0.48,0.48,0.37,0.02,0.39,0.37,0.39
1,2,Sławomir Abramowicz,POL,GK,Jagiellonia,20,12,12,1080,12.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,3,Francis Abu,GHA,MF,Cercle Brugge,24,7,7,554,6.2,...,0.00,0.16,0.16,0.00,0.16,0.02,0.03,0.05,0.02,0.05
3,4,Joshua Acheampong,ENG,DF,Chelsea,19,7,7,585,6.5,...,0.00,0.00,0.00,0.00,0.00,0.03,0.03,0.06,0.03,0.06
4,5,Elias Achouri,TUN,"FW,MF",FC Copenhagen,26,6,4,336,3.7,...,0.00,0.00,0.00,0.00,0.00,0.27,0.07,0.34,0.27,0.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,878,Aron Þrándarson,ISL,"MF,FW",KV,30,7,7,465,5.2,...,0.00,0.00,0.00,0.00,0.00,0.06,0.03,0.09,0.06,0.09
913,879,Pavle Đajić,BIH,"DF,FW",Borac Banja Luka,18,2,0,32,0.4,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
914,880,Ifet Đakovac,BIH,"MF,FW",TSC Bačka Top,27,6,5,440,4.9,...,0.41,0.20,0.61,0.41,0.61,0.48,0.20,0.68,0.28,0.48
915,881,Mateja Đorđević,SRB,DF,TSC Bačka Top,22,7,7,629,7.0,...,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.02,0.01,0.02


In [69]:
import os
# Définir le chemin de destination
dossier_destination = r"C:\Users\LENOVO\Documents\Projet Analyse du web\Final Document"
dfc_clean.to_csv(os.path.join(dossier_destination, "Conference_league_final_dataset.csv"), index=False)